## Импорт библиотек

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import json

# ВАЖНО ПРОЧИТАТЬ README И СКАЧАТЬ ДАТАСЕТ С ОБЛАКА, ССЫЛКА В README

## Функции используемые для создания данных для обучения

In [2]:
# Нахождение количества элементов в столбце != -1         
def heigh_of_col(mrx, index):
    column = mrx[:, index]
    mask = column != -1
    count_non_negative = np.sum(mask)
    return count_non_negative

# Нахождение количества элементов != -1 в строке, с наименьшим количеством элементов = -1 во всем массиве
def max_width_spinneret(matrix_array):
    
    # Находим индексы элементов, не равных -1, для каждой строки 
    non_negative_indices = (matrix_array != -1)                          
                                                                            
    # Считаем количество элементов, не равных -1, для каждой строки      
    count_non_negative_elements = np.sum(non_negative_indices, axis=1)   
                                                                            
    # Находим индекс строки с максимальным количеством элементов, не равных -1
    max_row_index = np.argmax(count_non_negative_elements)
    max_width = count_non_negative_elements[max_row_index]
    
    return max_width

# Нахождение количества элементов != -1 в строке
def width_row(matrix_array, row):
    # Находим индексы элементов, не равных -1, в выбранной строке
    non_negative_indices_in_selected_row = (matrix_array[row] != -1)

    # Считаем количество элементов, не равных -1, в выбранной строке
    count_non_negative_elements_in_selected_row = np.sum(non_negative_indices_in_selected_row)
    
    return count_non_negative_elements_in_selected_row

# Нахождение количества элементов != -1 до определенного элемента
def length_to_elements(matrix_array, row, column):
    selected_row_part = matrix_array[row, :column + 1]
    non_negative_indices_in_selected_row_part = (selected_row_part != -1)
    count_non_negative_elements_in_selected_row_part = np.sum(non_negative_indices_in_selected_row_part)
    
    return count_non_negative_elements_in_selected_row_part

# Данная функция позволяет обработать матрицу для составления датасета для обучения ML (В датасет включаются все точки, кроме находящихся у стенки труб)
def process_matrix(mrx, dataset):
    max_width = max_width_spinneret(mrx) 

    for row in range(1, 201):
        for col in range(0, 203):
            if mrx[row, col] != -1:
                submrx = mrx[row - 1 : row + 1, col - 1: col + 2]
                if submrx[0, 0] != -1 and submrx[0, 2] !=-1:
                    wdh_pr = width_row(mrx, row - 1)
                    
                    wdh_r = width_row(mrx, row)
                    lng = length_to_elements(mrx, row, col)
                    hgt_c = heigh_of_col(mrx, col)

 
                    
                    central = submrx[1, 1]
                    outer = []
                    d4_d1 = wdh_pr / max_width
                    d2_d1 = wdh_r / max_width
                    d3_d2 = lng / wdh_r

                    dyn_dy = (row + 1) / hgt_c
                    outer.append(row)
                    outer.append(col)
                    for x_offset, y_offset in [(0, 0), (0, 1), (0, 2)]:
                        outer_value = submrx[x_offset, y_offset]
                        outer.append(outer_value)

                    outer.append(d2_d1)
                    outer.append(d3_d2)
                    outer.append(d4_d1)
                    outer.append(dyn_dy)
                    outer.append(central)
                    dataset.append(outer)
# Данная функция позволяет обработать матрицу для составления датасета для обучения ML (В датасет включаются точки, находящиеся у стенки труб)
def process_matrix_wall(mrx, dataset):
    
    max_width = max_width_spinneret(mrx) 

    for row in range(1, 201):
        for col in range(0, 203):
            if mrx[row, col] != -1:
                submrx = mrx[row - 1 : row + 1, col - 1: col + 2]
                if submrx[0, 0] == -1 or submrx[0, 2] == -1:
                    wdh_pr = width_row(mrx, row - 1)
                    
                    wdh_r = width_row(mrx, row)
                    lng = length_to_elements(mrx, row, col)
                    hgt_c = heigh_of_col(mrx, col)                   
                    central = submrx[1, 1]
                    outer = []
                    d4_d1 = wdh_pr / max_width
                    d2_d1 = wdh_r / max_width
                    d3_d2 = lng / wdh_r

                    dyn_dy = (row + 1) / hgt_c
                    outer.append(row)
                    outer.append(col)
                    for x_offset, y_offset in [(0, 0), (0, 1), (0, 2)]:
                        outer_value = submrx[x_offset, y_offset]
                        outer.append(outer_value)

                    outer.append(d2_d1)
                    outer.append(d3_d2)
                    outer.append(d4_d1)
                    outer.append(dyn_dy)

                    outer.append(central)
                    dataset.append(outer)
                
                          

   

## Получение названий файлов датасета

In [3]:
datasets = []

for filename in os.listdir('CSV dataset'):
    datasets.append(f'CSV dataset/{filename}')
print(datasets)    
   

['CSV dataset/Dataset_0.csv', 'CSV dataset/Dataset_1.csv', 'CSV dataset/Dataset_3.csv', 'CSV dataset/Dataset_4.csv']


## Создание матриц для каждого файла из всех столбцов

In [4]:
def create_matrixes(columns, df, matrixes):
    for column in columns:
        matrix = pd.pivot_table(df, values=column, index='y', columns='x', fill_value=-1)
        mrx = np.flipud(matrix.to_numpy())
        matrixes.append(mrx)

In [13]:
step_x = 0.01
step_y = 0.01
columns = []
matrixes = []

df = pd.read_csv(datasets[0])
columns.append(list(df.columns[2:]))
print(columns)
print(len(columns[0]))
indexs = [0, 1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 15, 16, 20, 21, 22, 23, 24, 25, 26]
create_matrixes(columns[0][0:100], df, matrixes)

[['spf.U (m/s) @ v0=0.02_y2=3E-4', 'spf.U (m/s) @ v0=0.02_y2=3.5E-4', 'spf.U (m/s) @ v0=0.02_y2=4E-4', 'spf.U (m/s) @ v0=0.02_y2=4.5E-4', 'spf.U (m/s) @ v0=0.02_y2=5E-4', 'spf.U (m/s) @ v0=0.02_y2=5.5E-4', 'spf.U (m/s) @ v0=0.02_y2=6E-4', 'spf.U (m/s) @ v0=0.02_y2=6.5E-4', 'spf.U (m/s) @ v0=0.02_y2=7E-4', 'spf.U (m/s) @ v0=0.02_y2=7.5E-4', 'spf.U (m/s) @ v0=0.02_y2=8E-4', 'spf.U (m/s) @ v0=0.02_y2=8.5E-4', 'spf.U (m/s) @ v0=0.02_y2=9E-4', 'spf.U (m/s) @ v0=0.02_y2=9.5E-4', 'spf.U (m/s) @ v0=0.02_y2=0.001', 'spf.U (m/s) @ v0=0.02_y2=0.00105', 'spf.U (m/s) @ v0=0.02_y2=0.0011', 'spf.U (m/s) @ v0=0.02_y2=0.00115', 'spf.U (m/s) @ v0=0.02_y2=0.0012', 'spf.U (m/s) @ v0=0.02_y2=0.00125', 'spf.U (m/s) @ v0=0.02_y2=0.0013', 'spf.U (m/s) @ v0=0.02_y2=0.00135', 'spf.U (m/s) @ v0=0.02_y2=0.0014', 'spf.U (m/s) @ v0=0.02_y2=0.00145', 'spf.U (m/s) @ v0=0.02_y2=0.0015', 'spf.U (m/s) @ v0=0.04_y2=3E-4', 'spf.U (m/s) @ v0=0.04_y2=3.5E-4', 'spf.U (m/s) @ v0=0.04_y2=4E-4', 'spf.U (m/s) @ v0=0.04_y2=4.5E-4

In [15]:
columns[0][50]

'spf.U (m/s) @ v0=0.06_y2=3E-4'

# Обработка каждой матрицы и сохрание данных в файл (не у стенки)

In [16]:
dset = []
indexs = [0, 1, 2, 3, 4, 5, 6, 25, 26, 27, 28, 29, 30, 31, 50, 51, 52, 53, 54, 55, 56]
for i in indexs:
    process_matrix(matrixes[i], dset)
df = pd.DataFrame(dset)
columns = ['row','col','v1','v2','v3', 'd2_d1', 'd3_d2', 'd4_d1', 'dyn_dy', 'central']
df.columns = columns
df.to_csv('DS_no_walls1.csv', index=False)
df
    

,row,col,v1,v2,v3,d2_d1,d3_d2,d4_d1,dyn_dy,central
0,1,2,0.000000,0.000588,1.169852e-03,1.000000,0.009950,1.000000,0.011696,0.000588
1,1,3,0.000588,0.001170,1.746375e-03,1.000000,0.014925,1.000000,0.011696,0.001169
2,1,4,0.001170,0.001746,2.318073e-03,1.000000,0.019900,1.000000,0.011696,0.001745
3,1,5,0.001746,0.002318,2.882437e-03,1.000000,0.024876,1.000000,0.011696,0.002317
4,1,6,0.002318,0.002882,3.441727e-03,1.000000,0.029851,1.000000,0.011696,0.002881
...,...,...,...,...,...,...,...,...,...,...
684871,200,101,1.658868,1.706146,1.657182e+00,0.054726,0.545455,0.054726,1.000000,1.693299
684872,200,102,1.706146,1.657182,1.519700e+00,0.054726,0.636364,0.054726,1.000000,1.676117
684873,200,103,1.657182,1.519700,1.215456e+00,0.054726,0.727273,0.054726,1.000000,1.503707
684874,200,104,1.519700,1.215456,6.853274e-01,0.054726,0.818182,0.054726,1.000000,1.199637


# Обработка каждой матрицы и сохрание данных в файл (у стенки)

In [17]:
dset = []
# for mrx in matrixes:
#     process_matrix_wall(mrx, dset)
for i in indexs:
    process_matrix_wall(matrixes[i], dset)
df = pd.DataFrame(dset)
columns = ['row','col','v1','v2','v3', 'd2_d1', 'd3_d2', 'd4_d1', 'dyn_dy', 'central']
df.columns = columns
df.to_csv('DS_wall1.csv', index=False)
df

,row,col,v1,v2,v3,d2_d1,d3_d2,d4_d1,dyn_dy,central
0,1,1,-1.000000,0.000000e+00,0.000588,1.000000,0.004975,1.000000,0.011696,0.000000e+00
1,1,201,0.000588,0.000000e+00,-1.000000,1.000000,1.000000,1.000000,0.011696,0.000000e+00
2,2,1,-1.000000,0.000000e+00,0.000588,1.000000,0.004975,1.000000,0.017544,0.000000e+00
3,2,201,0.000588,0.000000e+00,-1.000000,1.000000,1.000000,1.000000,0.017544,0.000000e+00
4,3,1,-1.000000,0.000000e+00,0.000587,1.000000,0.004975,1.000000,0.023392,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...
7759,198,106,0.677562,2.662316e-15,-1.000000,0.054726,1.000000,0.054726,0.990050,2.328576e-15
7760,199,96,-1.000000,3.147379e-15,0.667760,0.054726,0.090909,0.054726,0.995025,3.001276e-15
7761,199,106,0.679777,2.328576e-15,-1.000000,0.054726,1.000000,0.054726,0.995025,2.612204e-15
7762,200,96,-1.000000,3.001276e-15,0.670089,0.054726,0.090909,0.054726,1.000000,3.629275e-15
